PY SAGAR -- Compared Logistic Regression and Random Forest -(With Confusion Matrix, HyperParameter Tuning,ROC Curve)

In [ ]:
#Importing Libraries
import numpy as np
import pandas as pd
from sklearn import linear_model, decomposition, datasets
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.metrics import r2_score,classification_report,f1_score,matthews_corrcoef,recall_score,plot_roc_curve
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score,mean_squared_error,mean_absolute_error
%matplotlib inline
import seaborn as sns

In [ ]:
#Loading the dataset
data = pd.read_csv('../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv')

#Print the first 5 rows of the dataframe.
data.head()

Here DEATH EVENT is Dependent Feauture

In [ ]:
X = data.drop('DEATH_EVENT',axis=1)
y = data['DEATH_EVENT']

In [ ]:
X.shape

In [ ]:
## Train test Split

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
log_reg = LogisticRegression()

In [ ]:
log_reg.fit(X_train,y_train)

In [ ]:
logr_pred = log_reg.predict(X_test)

In [ ]:
accuracyLR = accuracy_score(y_test,logr_pred)
accuracyLR

Got Accuracy of 86.6 % Accuracy in Logistics Regression

# Hyper parameter tuning of Logistics Regression

In [ ]:
log_reg = LogisticRegression()
grid = {"penalty" : ["l1", "l2"],"C" : np.arange(0,100,1)}
log_reg_cv = GridSearchCV(log_reg, grid, cv=3)
log_reg_cv.fit(X_train,y_train)

In [ ]:
print("Tuned hyperparameter n_estimators: {}".format(log_reg_cv.best_params_)) 
print("Best score: {}".format(log_reg_cv.best_score_))
print("Best Estimator: {}".format(log_reg_cv.best_estimator_))

In [ ]:
results_NB = pd.DataFrame(log_reg_cv.cv_results_['params'])
results_NB['test_score'] = log_reg_cv.cv_results_['mean_test_score']
results_NB

In [ ]:
#Performance Comparison for Logistics Regression
import matplotlib.pyplot as plt
for i in ['l1', 'l2']:
    temp = results_NB[results_NB['penalty'] == i]
    temp_average = temp.groupby('C').agg({'test_score': 'mean'})
    plt.plot(temp_average, marker = '.', label = i)
    
    
plt.legend()
plt.xlabel('C')
plt.ylabel("Mean CV Score")
plt.title("Logistic Regression Performance Comparison")
plt.show()

In [ ]:
model_LR = log_reg_cv.best_estimator_
model_LR.fit(X_train,y_train)
predictions_LR =  model_LR.predict(X_test)
print('\n')
print('Accuracy: ', accuracy_score(y_test,predictions_LR))
print('f1-score:', f1_score(y_test, predictions_LR))
print('Precision score: ', precision_score(y_test,predictions_LR))
print('Recall score: ', recall_score(y_test,predictions_LR))
print('MCC: ',matthews_corrcoef(y_test,predictions_LR) )
print('Mean Squared Error:', mean_squared_error(y_test, predictions_LR) ** 0.5)
print('Mean Absolute Error:', mean_absolute_error(y_test, predictions_LR) ** 0.5)
print('\n')
print(classification_report(y_test, predictions_LR))
print('\n')
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_test, predictions_LR), annot=True, ax = ax, fmt='g')
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')

Got Accuracy of 82 percent after hyper parameter tuning

In [ ]:

from sklearn.metrics import roc_curve, auc

In [ ]:
clf = LogisticRegression()
model=log_reg.fit(X_train,y_train)
pred_val = log_reg.predict(X_test)

### Compute ROC curve and ROC area for predictions on validation set
fpr, tpr, _ = roc_curve(y_test, pred_val)
roc_auc = auc(fpr, tpr)

### Plot
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='black', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Area under curve is 0.81,Performance of Model is Good

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print("accuracy: ", accuracy_score(y_test, predictions))

# Hyperparameter Tuning of Random Forest

In [ ]:
grid = {"n_estimators" : np.arange(0,200,2)}
rf = RandomForestClassifier()
rf_random = GridSearchCV(rf, grid, cv=3)
rf_random.fit(X_train,y_train)

In [ ]:
print(rf_random.best_params_)
print(rf_random.best_estimator_)

In [ ]:
results_NB = pd.DataFrame(rf_random.cv_results_['params'])
results_NB['test_score'] = rf_random.cv_results_['mean_test_score']
results_NB

In [ ]:
#NB Performance Comparison 
plt.plot(results_NB['n_estimators'], results_NB['test_score'], marker = '.') 
plt.xlabel('n_estimators')
plt.ylabel("Mean CV Score")
plt.title("NB Performance Comparison")
plt.show()

In [ ]:
model_RF = rf_random.best_estimator_
model_RF.fit(X_train,y_train)
predictions_RF =  model_RF.predict(X_test)
print('\n')
print('Accuracy: ', accuracy_score(y_test,predictions_RF))
print('f1-score:', f1_score(y_test, predictions_RF))
print('Precision score: ', precision_score(y_test,predictions_RF))
print('Recall score: ', recall_score(y_test,predictions_RF))
print('MCC: ',matthews_corrcoef(y_test,predictions_RF) )
print('Mean Squared Error:', mean_squared_error(y_test, predictions_RF) ** 0.5)
print('Mean Absolute Error:', mean_absolute_error(y_test, predictions_RF) ** 0.5)
print('\n')
print(classification_report(y_test, predictions_RF))
print('\n')
ax= plt.subplot()
sns.heatmap(confusion_matrix(y_test, predictions_RF), annot=True, ax = ax, fmt='g')
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')

In [ ]:

### Fit a sklearn classifier on train dataset and output probabilities
clf = RandomForestClassifier()
model=clf.fit(X_train, y_train)
pred_val = clf.predict(X_test)



### Compute ROC curve and ROC area for predictions on validation set
fpr, tpr, _ = roc_curve(y_test, pred_val)
roc_auc = auc(fpr, tpr)

### Plot
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='black',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='green', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

Area under curve is 0.87,Means model is good